In [45]:
import pandas as pd
import numpy as np

notes_file = 'NOTEEVENTS_md.csv'  # 'NOTEEVENTS_lg.csv'
# Load the data
df = pd.read_csv(notes_file)

doc = df.iloc[0]['text']
print(doc)

#for token in doc:
 #   print(token, token.pos_, token.dep_)

KeyError: 'text'

In [ ]:
# Understand what is in the csv file
df.info()